In [3]:
import os
import json
import requests
from pathlib import Path
from transformers import (
    AutoImageProcessor,
    TFVisionTextDualEncoderModel,
    BertTokenizer,
    )
import tensorflow as tf
import numpy as np

/home/enrico/univr/Tesi/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-10 09:19:42.593818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 09:19:42.768251: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 09:19:42.770217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 09:19:43.855365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
image_path_to_caption_file = 'image_path_to_caption_val.json'
images_dir = '/home/enrico/univr/Tesi/images'
image_path_to_caption = json.load(open(image_path_to_caption_file, 'r'))    
img_paths = list(image_path_to_caption.keys())
img_paths[0]

'galaxy-emperador-marble-tile-21838-1s.jpg'

In [5]:
image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def crop_to_square(image):
    height, width = tf.shape(image)[0], tf.shape(image)[1]
    if height > width:
        image = tf.image.crop_to_bounding_box(
            image, (height - width) // 2, 0, width, width)
    elif width > height:
        image = tf.image.crop_to_bounding_box(
            image, 0, (width - height) // 2, height, height)
    return image

In [7]:
def load_image(image_path, images_dir):
    image = tf.io.read_file(os.path.join(images_dir,image_path))
    image = tf.image.decode_image(
        image, channels=3, expand_animations=False)
    image = crop_to_square(image)
    image = tf.image.resize(
        image, list(image_processor.size.values()), method="bicubic", antialias=True)
    image = image / 255.0
    image = (image - image_processor.image_mean) / image_processor.image_std
    # Convert to channels-first
    image = tf.transpose(image, perm=[2, 0, 1])

    return image

In [8]:
images = np.stack([load_image(p, images_dir) for p in img_paths], axis=0)
images.shape

(424, 3, 224, 224)

In [9]:
# !saved_model_cli show --dir {'text_model/1'} --all

In [10]:
# !docker pull tensorflow/serving
# !docker run -d --name serving_base tensorflow/serving
# !docker cp ../trainer/saved_models serving_base:/models/clip_bert
# !docker commit --change "ENV MODEL_NAME clip_bert" serving_base clip_bert
# !docker kill serving_base
# !docker run -d -p 8501:8501 --name clip_bert clip_bert

In [1]:
# !docker run -p 8501:8501 \
#     --mount type=bind,source=/home/enrico/univr/Tesi/notebooks/serving_text_models,target=/models/clip_bert \
#     --mount type=bind,source=/home/enrico/univr/Tesi/notebooks/models.config,target=/models/models.config \
#     -t tensorflow/serving --model_config_file=/models/models.config

2023-10-10 07:19:08.554038: I tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2023-10-10 07:19:08.554086: I tensorflow_serving/model_servers/server_core.cc:594]  (Re-)adding model: clip_bert
2023-10-10 07:19:08.754115: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: clip_bert version: 16}
2023-10-10 07:19:08.754145: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: clip_bert version: 16}
2023-10-10 07:19:08.754157: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: clip_bert version: 16}
2023-10-10 07:19:08.756029: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /models/clip_bert/16
2023-10-10 07:19:08.855467: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: clip_bert version: 15}
2023-10-10 07:19:08.855499: I tensorflow_serving/core/loa

In [10]:
# for model_id in range(10,17):
#     model = TFVisionTextDualEncoderModel.from_vision_text_pretrained(
#         text_model_name_or_path='bert-base-uncased',vision_model_name_or_path='google/vit-base-patch16-224')
#     model.load_weights(f'../trainer/saved_models/{model_id}')

#     tf.keras.models.save_model(
#         model.text_model,
#         f'serving_text_models/{model_id}',
#         overwrite=True,
#         include_optimizer=True
#     )

In [13]:
def meanReciprocalRank(model_id, k, images, img_paths, image_path_to_caption):
    rank_list = []
    model = TFVisionTextDualEncoderModel.from_vision_text_pretrained(text_model_name_or_path='bert-base-uncased',vision_model_name_or_path='google/vit-base-patch16-224')
    model.load_weights(f'../trainer/saved_models/{model_id}')
    if os.path.exists(f'img_features/{model_id}.npy'):
        image_features = np.load(f'img_features/{model_id}.npy')
    else:
        Path("img_features").mkdir(parents=True, exist_ok=True)
        image_features = model.get_image_features(images)
        np.save(f'img_features/{model_id}',image_features)
    for index,list_elem in enumerate(img_paths):
        batch = tokenizer(image_path_to_caption.get(list_elem)[0])
        batch = dict(batch)
        batch = [batch]
        input_data = {"instances": batch}
        r = requests.post(f"http://localhost:8501/v1/models/clip_bert/versions/{model_id}:predict", data=json.dumps(input_data))
        result = json.loads(r.text)["predictions"][0]
        pooled_output = model.text_projection(tf.expand_dims(result['pooler_output'],0))
        logits = np.dot(image_features, pooled_output.numpy().T)
        probs = tf.nn.softmax(logits, axis=0)
        rank = 1 + np.argsort(probs, axis=0).flatten()[::-1].tolist().index(index)
        if rank > k:
            rank_list.append(0)
        else:
            rank_list.append(1/rank)
    return f'MRR @ {k} for model {model_id} is {np.mean(rank_list)}'
# meanReciprocalRank(4,10,images,img_paths,image_path_to_caption)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'c

'MRR @ 10 for model 4 is 0.06364461665169212'

In [13]:
mrr=[]
for i in range(10,17):
    mrr.append(meanReciprocalRank(i, 10, images, img_paths, image_path_to_caption))
mrr

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerN

['MRR @ 10 for model 10 is 0.012522461814914645',
 'MRR @ 10 for model 11 is 0.07339398023360287',
 'MRR @ 10 for model 12 is 0.12082958969751421',
 'MRR @ 10 for model 13 is 0.12725273285414795',
 'MRR @ 10 for model 14 is 0.12728361784965558',
 'MRR @ 10 for model 15 is 0.06280604222821204',
 'MRR @ 10 for model 16 is 0.032708146151542376']